### Using CNN to predict vorticity from temperature and kinetic energy data

I've implemented a convolutional neural network with five hidden layers which takes an input 75x75 grid of temperature or kinetic energy readings and predicts the vorticity in those regions. 

The dataset used to train the network consists of CMEMS data from 1993 to 2016 of various oceans. Each 600x600 grid of data was split into 64 75x75 grids where each pixel represents 1/12 degrees as shown in the image below.

<img src="full_grid_KE.png" alt="strain&deformationtype" width="300"/>

Doing this, I was able to create a dataset with 142,000 input output image pairs which I used to train the network. Below, I show my code as well as the results of the trained network.

In [ ]:
# importing relevant packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, Reshape
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from sklearn.experimental import enable_iterative_imputer
from tensorflow.keras.models import load_model
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn import model_selection
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
import xarray as xr
import pickle

Next I wrote a library of functions which will be useful in the future 

In [ ]:
# takes a vector of x-locations and a vector of y-locations 
# and calculates the slope (dy/dx) at each x-location.
def slope(y, x):
    l = np.size(y)
    s = np.zeros(l)
    s[0] = (y[1] - y[0]) / (x[1] - x[0])
    s[l - 1] = (y[l - 1] - y[l - 2]) / (x[l - 1] - x[l - 2])
    for i in range(1, l - 1, 1):
        s[i] = (y[i + 1] - y[i - 1]) / (x[i + 1] - x[i - 1])
    return s

# calculates curl of a vector F on a 2D rectangular grid
def curl(x, y, Fx, Fy):
    dFy_dx = np.zeros((len(y), len(x)))
    dFx_dy = np.zeros((len(y), len(x)))

    for iy in range(len(y)):
        dFy_dx[iy, :] = slope(np.ravel(Fy[iy, :]), x)

    for ix in range(len(x)):
        dFx_dy[:, ix] = slope(np.ravel(Fx[:, ix]), y)

    return dFy_dx - dFx_dy

# calculates the vorticity when given the ocean data from the get_data function
def vorticity(data):
    lons, lats, vx, vy = data[0], data[1], data[2], data[3]
    w, h, t_total = np.shape(vx)[1], np.shape(vx)[2], np.shape(vx)[0]
    vor = np.zeros((t_total, w, h), dtype=np.float32)
    for i in range(t_total):
        vor[i, :, :] = curl(lons, lats, vx[i, :], vy[i, :])
    return vor

# takes the .nc filename and outputs its data as numpy arrays
def get_data(filename, n):
    ds = xr.open_dataset(filename)

    d, time = ds.depth, ds.time
    lat, long = ds.latitude, ds.longitude

    if n == 1:
        v_x, v_y = ds.uo, ds.vo
        temp = ds.thetao
    else:
        v_x, v_y = ds.u, ds.v
        temp = ds.temperature

    temp = temp.values
    lats, lons = lat.values, long.values
    vx, vy = v_x.values, v_y.values

    w = np.shape(vx[0, :])[1]
    h = np.shape(vx[0, :])[2]
    t_total = len(time.values)
    w, h = np.shape(vx[0, :])[1], np.shape(vx[0, :])[2]
    vx, vy, temp = vx.reshape((t_total, w, h)), vy.reshape((t_total, w, h)), temp.reshape((t_total, w, h))
    data = [lons, lats, vx, vy, temp, time]
    return data

# takes a 3d matrix and splits each submatrix into 4 submatricies for each iteration
# input is an NxMxM matrix and output is N'xM'xM' matrix 
# where N' = (4^iterations)*N and M' = M/(2^iterations)
def split_data(A, iterations):
    def split_in_four(A):
        shape = np.shape(A)
        d, l, w = shape[0], shape[1], shape[2]
        div = int(l/2)
        upperLeft  = A[0:d,0:div,0:div]
        upperRight = A[0:d,0:div,div:2*div]
        lowerLeft  = A[0:d,div:2*div,0:div]
        lowerRight = A[0:d,div:2*div,div:2*div]
        combined   = np.concatenate((upperLeft, upperRight, lowerLeft, lowerRight))
        return combined
    for i in range(iterations):
        A = split_in_four(A)
        if np.shape(A)[2] < 2:
            break
    return A

# fills NaN values with mean of each column
def fill_nans_simple(A):
    A0 = np.empty(np.shape(A))
    for i in range(len(A)):
        imp = SimpleImputer()
        imp.fit(A[i, :])
        A0[i, :] = imp.transform(A[i, :])
    return A0

# fills NaN values by modelling each feature as a function of missing futures
# gives better results than simple imputer but much more computationally heavy
def fill_nans_iterative(A):
    A0 = np.empty(np.shape(A))
    for i in range(len(A)):
        imp = IterativeImputer()
        imp.fit(A[i, :])
        A0[i, :] = imp.transform(A[i, :])
    return A0

# takes an n-dimensional array and normalizes its values to [0, 1]
def normalize(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))

# function for plotting the data. set plot_quiver = 1 to plot vector field,
# set plot_vorticity, plot_temperature, or plot_KE to 1 and the others to 0 to
# plot the desired variable
def plot_data(t, data, plot_quiver,
              plot_vorticity, plot_temperature, plot_KE):
    
    skip = 5 # skips data points when plotting vector field for better visualisation
    lons, lats, vx, vy, time, T = data[0], data[1], data[2], data[3], data[4], data[5]
    
    w, h = np.shape(vx)[1], np.shape(vx)[2]
    vx_s, vy_s = vx[t,:].reshape((w,h)), vy[t,:].reshape((w,h))
    time = time[t]
    
    if plot_vorticity:
        if not plot_temperature:
            vor = curl(lons, lats, vx_s, vy_s)   # calculating vorticity
           # vmin = np.nanmin(vor)/2
           # vmax = np.nanmax(vor)/2
            vmin = -5.5
            vmax = 5.5
            label = 'Vorticity'
        else:
            vor = T[t,:]
            vmin = np.nanmin(vor)
            vmax = np.nanmax(vor)
            label = 'Temperature'
    if plot_KE:
        vor = np.sqrt(vx_s**2 + vy_s**2)/2
        vmin = np.nanmin(vor)
        vmax = np.nanmax(vor)
        label = 'Kinetic Energy'
    
    fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree(
        central_longitude=0.0, globe=None)})
    fig.set_size_inches([30,15])

    ax.set_global()
    ax.stock_img()
    ax.text(0.55, 0.95, str(time)[0:10], 
        transform=ax.transAxes, ha="right", color='black', 
        bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))
    
    if plot_vorticity:   
        plt.contourf(lons, lats, vor, 99, levels=np.linspace(-5.5, 5.5, 100, endpoint=True),
                     vmin = vmin, vmax = vmax, transform=ccrs.PlateCarree())
        
        cbaxes = inset_axes(ax, width="3.5%", height="45%", loc=2) 
        cbar = plt.colorbar(cax=cbaxes, ticks=np.arange(-5,6,1));
        cbar.ax.set_ylabel(label, rotation=270, labelpad=10)
        
    if plot_quiver:
        ax.quiver(lons[::skip], lats[::skip], vx_s[::skip, ::skip], vy_s[::skip, ::skip],
            transform=ccrs.PlateCarree(), width=0.002)
    
    ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=ccrs.PlateCarree())
    plt.savefig('Images/' + 'img-'+str(t)+'.jpeg', bbox_inches = 'tight', pad_inches = 0, dpi=100)
    plt.clf()


Next I downloaded the data from CMEMs in increments (due to the 1 GB download limit). I then wrote a script that reads the .nc files and converts them into numpy arrays, splits them into smaller grids, normalizes their values, and fills all the NaN values then appends and saves them.

In [ ]:
filenames = ['nc_data/1.nc', 'nc_data/2.nc', 'nc_data/4.nc', 
             'nc_data/5.nc', 'nc_data/6.nc', 'nc_data/7.nc']
file_len = len(filenames)
split = 3
x, y, z = int((4**split)*312), int(601/(2**split)), int(601/(2**split))
vor_total, KE_total, T_total = (np.empty((file_len*x, y, z), dtype=np.float32)
                                , np.empty((file_len*x, y, z), dtype=np.float32)
                                , np.empty((file_len*x, y, z), dtype=np.float32))
for i in range(file_len):
    filename = filenames[i]
    data = get_data(filename, 1)
    lons, lats, vx, vy, T, time = data
    KE = (vx**2 + vy**2)
    vx, vy, lons, lats, time = None, None, None, None, None
    vor = vorticity(data)
    data = None
    vor_split = split_data(vor, split)
    vor = None
    KE_split = split_data(KE, split)
    KE = None
    T_split = split_data(T, split)
    T = None
    vor_total[i*x:(i+1)*x,0:y, 0:z] = vor_split
    vor_split = None
    KE_total[i*x:(i+1)*x,0:y, 0:z] = KE_split
    KE_split = None
    T_total[i*x:(i+1)*x,0:y, 0:z] = T_split
    print(np.shape(T_total))
    print(np.shape(T_split))
    print(i)
    T_split = None

vor_total = normalize(fill_nans_simple(vor_total))
T_total = normalize(fill_nans_simple(T_total))
KE_total = normalize(fill_nans_simple(KE_total))
np.save('vorticity', vor_total)
np.save('temperature', T_total)
np.save('kinetic energy', KE_total)

Now I write and train my neural network then save the trained model. The data is split 80:10:10 where 80% of the 140,000 images are used for training and 10% are used for validation and 10% are used for testing. This is done for both temperature -> vorticity and KE -> vorticity. Each was trained for 100 epochs and took approximately 12 hours to train.

In [ ]:
ke = np.load('npy_data_files/kinetic energy.npy')  # can also use temperature
vor = np.load('npy_data_files/vorticity.npy')

imsize = np.shape(ke)[1]

# splitting data into train/validation sets
train_X, validation_X, train_Y, validation_Y = model_selection.train_test_split(ke, vor, test_size=0.20)
x1, y1, z1 = np.shape(train_X)
x2, y2, z2 = np.shape(validation_X)
train_X, train_Y = train_X.reshape((x1, y1, z1, 1)), train_Y.reshape((x1, y1, z1, 1))
validation_X, validation_Y = validation_X.reshape((x2, y2, z2, 1)), validation_Y.reshape((x2, y2, z2, 1))

batch_size = 64
epochs = 100

print('x_train shape:', train_X.shape)
print(train_X.shape[0], 'train samples')
print(validation_X.shape[0], 'validation samples')

# building the network
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=(imsize, imsize, 1), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Conv2D(32, (3, 3), padding='same', dilation_rate=1))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Conv2D(64, (3, 3), padding='same', dilation_rate=2))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Conv2D(128, (3, 3), padding='same', dilation_rate=4))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Conv2D(256, (3, 3),  padding='same', dilation_rate=4))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))

model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Dense(1))

model.compile(loss=keras.losses.mean_squared_error,  
              optimizer=keras.optimizers.Adam(), verbose=1,  metrics=['accuracy'])
model.summary()

# training the network and saving the trained network and its training history
model_train = model.fit(train_X, train_Y, batch_size=batch_size, epochs=epochs,
                        validation_data=(validation_X, validation_Y), shuffle=True)
model.save('my_model.h5')
history_dict = model_train.history
f = open('history.pckl', 'wb')
pickle.dump(history_dict, f)
f.close()


After training the model was then given unseen data and made to predict vorticity and the results were compared with actual voriticty values. You can check out the results [here](https://1drv.ms/u/s!AmK5uxmgpTOMgct66dv0Xh2ap7QXxA?e=QNSJ48). Each image is made up of three images, the input KE or Temperature, the predicted vorticity from the neural network, and the actual vorticity. As you can see, the neural network does a good job predicting the vorticity, despite only consisting of 6 layers and ~500,000 parameters.

In [ ]:
# use trained model to predict vorticity based on temperature/kinetic energy

model = load_model('my_model_KE.h5')    # can change with temperature
KE = np.load('kinetic energy test.npy') # can change with temperature
vor = np.load('vorticity test.npy')

fig = plt.figure()
plt.rcParams['figure.figsize'] = [10, 5]
n = 30

for i in range(n):
    KE0 = KE[i*int(5000/n), :]
    vor0 = vor[i*int(5000/n), :]
    Y = model.predict(KE0.reshape(1, 75, 75, 1))
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.set_title('Input KE', size=8)
    ax1.imshow(KE[i*int(5000/n), :], interpolation='Gaussian')
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 3, 2)
    ax2.set_title('Predicted Output Vorticity', size=8)
    ax2.imshow(Y.reshape(75, 75), interpolation='Gaussian')
    ax2.axis('off')
    ax3 = fig.add_subplot(1, 3, 3)
    ax3.set_title('Real Vorticity', size=8)
    ax3.imshow(vor0, interpolation='Gaussian')
    ax3.axis('off')
    fig.savefig('image results/' + str(i+1), bbox_inches='tight', dpi=200)
    fig.clf()